<a href="https://colab.research.google.com/github/NikolaLjubicicc/Gender_Age_Emotion_Rec/blob/main/IT44_2021_PPPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!unzip -q Faces_AgeGenderEmotionDetection.zip -d FacesAgeGenderEmotionDetection

In [3]:
!pip install mtcnn opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.1 MB/s eta 0:00:00


In [4]:
import json, pandas as pd, os
import tensorflow as tf
from tensorflow.keras import Model, layers, Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from mtcnn import MTCNN
import dlib, cv2, numpy as np
from glob import glob
from tensorflow.keras.layers import Resizing, RandomCrop

In [6]:
with open("/content/labels_model_2025-07-02-03-12-48.json") as f:
    train_recs = json.load(f)
with open("/content/labels_model_val_2025-07-03-03-31-03.json") as f:
    val_recs   = json.load(f)
BASE = "/content/FacesAgeGenderEmotionDetection/data"

file_map = {}
for split in ("train","val"):
    split_dir = os.path.join(BASE, split)
    for act in os.listdir(split_dir):
        act_dir = os.path.join(split_dir, act)
        if not os.path.isdir(act_dir):
            continue
        for img in os.listdir(act_dir):
            file_map[img] = os.path.join(act_dir, img)

A_G_E = []
for rec in train_recs:
    fn = rec["image"]
    full_path = file_map.get(fn)
    if full_path is None:
        raise FileNotFoundError(f"'{fn}' nije pronađen u pod-folderima!")
    age, gender, emotion = [s.strip() for s in rec["annotations"][0].split(",")]
    A_G_E.append({
      "filepath": full_path,
      "age": int(age),
      "gender": 0 if gender=="M" else 1,
      "emotion": emotion.lower(),
      "split": "train"
    })
for rec in val_recs:
    fn = rec["image"]
    full = file_map.get(fn)
    if full is None:
        raise FileNotFoundError(fn)
    age, gender, emotion = [s.strip() for s in rec["annotations"][0].split(",")]
    A_G_E.append({
        "filepath": full,
        "age": int(age),
        "gender": 0 if gender=="M" else 1,
        "emotion": emotion.lower(),
        "split": "val"
    })

frame = pd.DataFrame(A_G_E)


train_df = frame[frame.split=="train"].reset_index(drop=True)
val_df   = frame[frame.split=="val"].reset_index(drop=True)

# Konstante
emotions    = sorted(frame["emotion"].unique())
em2idx      = {e:i for i,e in enumerate(emotions)}
num_age = 80
age_bin = 10 #
age_groups = (num_age + age_bin - 1) // age_bin
num_gender  = 2
num_emotion = len(emotions)
img_size    = 64
BATCH       = 16
detector = MTCNN()

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.4),
    layers.RandomZoom(height_factor=0.4, width_factor=0.2),
    layers.RandomContrast(0.2),
    layers.RandomContrast(factor=0.3),
    layers.RandomTranslation(height_factor=0.3, width_factor=0.3),

], name="data_augmentation")

def detect_and_crop(path):
    # path je bajt string, konvertujemo u str
    path = path.numpy().decode('utf-8')
    # Ucitavanje
    img_bgr = cv2.imread(path)
    if img_bgr is None:
        # Pravi se crna slika da ne izbacuje gresku, ako se slika ne ucita
        img_bgr = np.zeros((img_size, img_size, 3), dtype=np.uint8)

    img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB) # Konvertovanje u RGB

    # Detektovanje lica
    results = detector.detect_faces(img)
    if results:
        x, y, w, h = results[0]['box']
        x, y = max(0, x), max(0, y)
        face = img[y:y+h, x:x+w]
    else:
        # Ako nema detekcije, koristi celu sliku
        face = img

    # resize
    face = cv2.resize(face, (img_size, img_size))

    face = face.astype(np.float32) / 255.0
    return face
def parse_fn(path, age, gender, emo):
    # učitavanje i resize
    img = tf.py_function(func=detect_and_crop,inp=[path], Tout=tf.float32)
    img.set_shape((img_size, img_size, 3))
    age_group = age // age_bin

    age_group = tf.clip_by_value(age_group, 0, age_groups-1)

    # Enkodiranje tabela
    y_age     = tf.one_hot(age_group, age_groups)
    y_gender  = tf.one_hot(gender, num_gender)
    y_emotion = tf.one_hot(emo,    num_emotion)

    return img, {"age": y_age, "gender": y_gender, "emotion": y_emotion}


def create(sub_df, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((
        sub_df["filepath"].values,
        sub_df["age"].values,
        sub_df["gender"].values,
        sub_df["emotion"].map(em2idx).values
    ))
    if shuffle and len(sub_df) > 1:
        ds = ds.shuffle(len(sub_df))
    ds = ds.map(parse_fn, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH).prefetch(tf.data.AUTOTUNE)
    return ds

# Splitovanje podataka u train i val setove
train_ds = create(train_df, shuffle=True).repeat()
val_ds   = create(val_df,   shuffle=False)

def cnn(input_shape=(img_size,img_size,3)):
    x_in = layers.Input(shape=input_shape)
    x = data_augmentation(x_in)
    # Blok 1
    x = layers.Conv2D(16, 3, padding="same")(x_in)
    x = layers.BatchNormalization()(x) #Normalizovanje
    x = layers.Activation("relu")(x) #Primena ReLu aktivacionog sloja
    x = layers.MaxPool2D(2)(x)   #MaxPooling sazimanje informacija, smanjenje broja paramaetara
    # Blok 2
    x = layers.Conv2D(32, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPool2D(2)(x)
    # Blok 3
    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPool2D(2)(x)
    # Blok 4
    x = layers.Conv2D(128, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPool2D(2)(x)

    x = layers.GlobalAveragePooling2D()(x)

    # Gusti sloj
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.5)(x)  # <- kada sam povecao sa 0.5 na .07 dobijen val loss oko 6.5

    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)

    # Izlazi
    age_output = layers.Dense(age_groups, activation="softmax", name="age")(x)
    gender_output = layers.Dense(num_gender, activation="softmax", name="gender")(x)
    emotion_output = layers.Dense(num_emotion, activation="softmax", name="emotion")(x)

    return Model(inputs=x_in, outputs=[age_output, gender_output, emotion_output])

model = cnn()
model.compile(
    optimizer=Adam(1e-4),
    loss={"age":"categorical_crossentropy", "gender":"categorical_crossentropy",
          "emotion":"categorical_crossentropy"},
    loss_weights={
    "age": 1.0, #sa 1.0 na 0.5 nije doprinelo poboljsanju rezultata
    "gender": 1.0,
    "emotion":1.0
  },
    metrics={"age":"accuracy","gender":"accuracy","emotion":"accuracy"}
)


callbacks = [
    tf.keras.callbacks.ModelCheckpoint("model.h5", save_best_only=True, monitor="val_loss"),
    tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=35,
    steps_per_epoch=len(train_df)//BATCH,
    validation_steps=len(val_df)//BATCH,
    callbacks=callbacks
)


Epoch 1/35
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - age_accuracy: 0.0479 - age_loss: 2.4807 - emotion_accuracy: 0.3023 - emotion_loss: 1.9404 - gender_accuracy: 0.5900 - gender_loss: 0.6766 - loss: 5.0977

5/5 ━━━━━━━━━━━━━━━━━━━━ 63s 10s/step - age_accuracy: 0.0503 - age_loss: 2.4830 - emotion_accuracy: 0.2998 - emotion_loss: 1.9437 - gender_accuracy: 0.5875 - gender_loss: 0.6759 - loss: 5.1026 - val_age_accuracy: 0.0000e+00 - val_age_loss: 2.0750 - val_emotion_accuracy: 0.2500 - val_emotion_loss: 1.6074 - val_gender_accuracy: 0.1250 - val_gender_loss: 0.6992 - val_loss: 4.3817
Epoch 2/35
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - age_accuracy: 0.1033 - age_loss: 2.5303 - emotion_accuracy: 0.2188 - emotion_loss: 1.6096 - gender_accuracy: 0.5073 - gender_loss: 0.7332 - loss: 4.8639

5/5 ━━━━━━━━━━━━━━━━━━━━ 40s 10s/step - age_accuracy: 0.1083 - age_loss: 2.5046 - emotion_accuracy: 0.2156 - emotion_loss: 1.6230 - gender_accuracy: 0.5139 - gender_loss: 0.7297 - loss: 4.8475 - val_age_accuracy: 0.1250 - val_age_loss: 2.0740 - val_emotion_accuracy: 0.2500 - val_emotion_loss: 1.6026 - val_gender_accuracy: 0.5000 - val_gender_loss: 0.6930 - val_loss: 4.3697
Epoch 3/35
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - age_accuracy: 0.0835 - age_loss: 2.3178 - emotion_accuracy: 0.3525 - emotion_loss: 1.5921 - gender_accuracy: 0.4561 - gender_loss: 0.8580 - loss: 4.7334

5/5 ━━━━━━━━━━━━━━━━━━━━ 38s 9s/step - age_accuracy: 0.0896 - age_loss: 2.3173 - emotion_accuracy: 0.3537 - emotion_loss: 1.5896 - gender_accuracy: 0.4712 - gender_loss: 0.8527 - loss: 4.7254 - val_age_accuracy: 0.1250 - val_age_loss: 2.0754 - val_emotion_accuracy: 0.2500 - val_emotion_loss: 1.5978 - val_gender_accuracy: 0.8750 - val_gender_loss: 0.6775 - val_loss: 4.3508
Epoch 4/35
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - age_accuracy: 0.2184 - age_loss: 1.9634 - emotion_accuracy: 0.3119 - emotion_loss: 1.5934 - gender_accuracy: 0.5203 - gender_loss: 0.7875 - loss: 4.3378

5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - age_accuracy: 0.2176 - age_loss: 1.9796 - emotion_accuracy: 0.3088 - emotion_loss: 1.5936 - gender_accuracy: 0.5247 - gender_loss: 0.7814 - loss: 4.3482 - val_age_accuracy: 0.1875 - val_age_loss: 2.0711 - val_emotion_accuracy: 0.2500 - val_emotion_loss: 1.5916 - val_gender_accuracy: 0.8750 - val_gender_loss: 0.6659 - val_loss: 4.3286
Epoch 5/35
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - age_accuracy: 0.2014 - age_loss: 2.0754 - emotion_accuracy: 0.3417 - emotion_loss: 1.5930 - gender_accuracy: 0.5074 - gender_loss: 0.7744 - loss: 4.4502

...

5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - age_accuracy: 0.5135 - age_loss: 1.4240 - emotion_accuracy: 0.4184 - emotion_loss: 1.1778 - gender_accuracy: 0.5910 - gender_loss: 0.6829 - loss: 3.2679 - val_age_accuracy: 0.3125 - val_age_loss: 2.0313 - val_emotion_accuracy: 0.3750 - val_emotion_loss: 1.3936 - val_gender_accuracy: 0.8750 - val_gender_loss: 0.5257 - val_loss: 3.9506
Epoch 27/35
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step - age_accuracy: 0.3639 - age_loss: 1.7105 - emotion_accuracy: 0.5313 - emotion_loss: 1.3182 - gender_accuracy: 0.5966 - gender_loss: 0.6659 - loss: 3.6523

5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - age_accuracy: 0.3744 - age_loss: 1.6931 - emotion_accuracy: 0.5317 - emotion_loss: 1.3108 - gender_accuracy: 0.6016 - gender_loss: 0.6647 - loss: 3.6257 - val_age_accuracy: 0.3125 - val_age_loss: 2.0287 - val_emotion_accuracy: 0.3750 - val_emotion_loss: 1.3855 - val_gender_accuracy: 0.8750 - val_gender_loss: 0.5186 - val_loss: 3.9328
Epoch 28/35
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - age_accuracy: 0.3114 - age_loss: 1.7280 - emotion_accuracy: 0.5243 - emotion_loss: 1.1583 - gender_accuracy: 0.6208 - gender_loss: 0.6866 - loss: 3.5529

5/5 ━━━━━━━━━━━━━━━━━━━━ 19s 5s/step - age_accuracy: 0.3172 - age_loss: 1.7244 - emotion_accuracy: 0.5191 - emotion_loss: 1.1630 - gender_accuracy: 0.6196 - gender_loss: 0.6830 - loss: 3.5491 - val_age_accuracy: 0.3750 - val_age_loss: 2.0318 - val_emotion_accuracy: 0.3750 - val_emotion_loss: 1.3784 - val_gender_accuracy: 0.8750 - val_gender_loss: 0.5119 - val_loss: 3.9221
Epoch 29/35
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - age_accuracy: 0.3981 - age_loss: 1.6335 - emotion_accuracy: 0.5943 - emotion_loss: 1.0088 - gender_accuracy: 0.5570 - gender_loss: 0.7458 - loss: 3.3807

5/5 ━━━━━━━━━━━━━━━━━━━━ 21s 5s/step - age_accuracy: 0.4007 - age_loss: 1.6350 - emotion_accuracy: 0.5841 - emotion_loss: 1.0182 - gender_accuracy: 0.5597 - gender_loss: 0.7440 - loss: 3.3878 - val_age_accuracy: 0.2500 - val_age_loss: 2.0404 - val_emotion_accuracy: 0.3750 - val_emotion_loss: 1.3727 - val_gender_accuracy: 0.8750 - val_gender_loss: 0.5071 - val_loss: 3.9202
Epoch 30/35
5/5 ━━━━━━━━━━━━━━━━━━━━ 19s 5s/step - age_accuracy: 0.4088 - age_loss: 1.6046 - emotion_accuracy: 0.5564 - emotion_loss: 1.0984 - gender_accuracy: 0.7995 - gender_loss: 0.4970 - loss: 3.2011 - val_age_accuracy: 0.1250 - val_age_loss: 2.0513 - val_emotion_accuracy: 0.3750 - val_emotion_loss: 1.3677 - val_gender_accuracy: 0.8750 - val_gender_loss: 0.5062 - val_loss: 3.9252
Epoch 31/35
5/5 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - age_accuracy: 0.5163 - age_loss: 1.3676 - emotion_accuracy: 0.5212 - emotion_loss: 1.1209 - gender_accuracy: 0.6613 - gender_loss: 0.6648 - loss: 3.1534 - val_age_accuracy: 0.1250 - val_a

5/5 ━━━━━━━━━━━━━━━━━━━━ 21s 5s/step - age_accuracy: 0.3372 - age_loss: 1.5379 - emotion_accuracy: 0.5698 - emotion_loss: 1.0076 - gender_accuracy: 0.7880 - gender_loss: 0.4957 - loss: 3.0353 - val_age_accuracy: 0.2500 - val_age_loss: 2.0652 - val_emotion_accuracy: 0.3750 - val_emotion_loss: 1.3432 - val_gender_accuracy: 0.8750 - val_gender_loss: 0.4993 - val_loss: 3.9077
